[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/viincci/Viincci-RAG/blob/main/viincci_rag/examples/test_examples.ipynb)

# Viincci-RAG — Minimal Examples and Safe SerpAPI Mock

Run this notebook in **Google Colab** (link above) or locally with Jupyter.

**Features:**
- Backwards-compatible imports (viincci_rag → V4 fallback)
- SerpAPI mock to avoid using credits while testing
- Multiple domain examples (botany, literature, carpentry)
- Safe fallback generation when library unavailable

# Viincci-RAG — Minimal Examples and Safe SerpAPI Mock

This notebook provides a small, safe set of example cells to:

- Prefer imports from `viincci_rag` (new layout) and fall back to the legacy `V4` package when needed (backwards compatible).
- Demonstrate a SerpAPI mock so you can run examples without consuming real SerpAPI credits (useful for the free 250 credits/month constraint).
- Provide tiny generation examples across several domains using either the real library (if installed) or a harmless fallback that stitches mock research results together.

Notes: Run cells sequentially. If you have the real library installed and a valid SerpAPI key configured, the notebook will attempt to use it; otherwise it uses safe mocks.

## Installation (Optional - uncomment if running in fresh Colab)

If you're running in Google Colab and don't have the library installed:

In [ ]:
# Installation & GitHub Test: Ensure viincci_rag is available
import subprocess
import sys
import os

print("=" * 70)
print("VIINCCI-RAG INSTALLATION & GITHUB TEST")
print("=" * 70)

# Step 1: Check if already installed
print("\n📦 Step 1: Checking current installation...")
try:
    import viincci_rag
    print(f"✅ viincci_rag is already installed")
    print(f"   Location: {viincci_rag.__file__}")
    ALREADY_INSTALLED = True
except ImportError:
    print("⚠️  viincci_rag not currently installed - will install now")
    ALREADY_INSTALLED = False

# Step 2: Install locally if not already installed
if not ALREADY_INSTALLED:
    print("\n? Step 2: Installing viincci_rag locally...")
    notebook_dir = os.getcwd()
    repo_root = os.path.abspath(os.path.join(notebook_dir, '../..'))
    
    try:
        result = subprocess.run(
            [sys.executable, '-m', 'pip', 'install', '--quiet', '--no-cache-dir', '-e', repo_root],
            capture_output=True,
            text=True,
            timeout=60
        )
        
        if result.returncode == 0:
            print(f"✅ Local installation successful!")
            print(f"   Source: {repo_root}")
            # Reload modules
            import importlib
            import sys
            for mod in list(sys.modules.keys()):
                if 'viincci_rag' in mod or 'V4' in mod:
                    del sys.modules[mod]
        else:
            print(f"⚠️  Local installation had issues: {result.stderr or result.stdout}")
    except subprocess.TimeoutExpired:
        print("⚠️  Local installation timed out")
    except Exception as e:
        print(f"⚠️  Local installation error: {e}")

# Step 3: Test fresh GitHub install (optional - for validation)
print("\n" + "=" * 70)
print("📥 GITHUB INSTALLATION TEST (Fresh Environment)")
print("=" * 70)
print("   Testing: pip install git+https://github.com/viincci/Viincci-RAG.git")

import tempfile
import shutil

test_env_dir = tempfile.mkdtemp(prefix="viincci_test_")
print(f"   Test directory: {test_env_dir}")

try:
    # Install in a fresh pip context
    result = subprocess.run(
        [sys.executable, '-m', 'pip', 'install', '--quiet', '--no-cache-dir',
         'git+https://github.com/viincci/Viincci-RAG.git'],
        capture_output=True,
        text=True,
        timeout=120
    )
    
    if result.returncode == 0:
        print("✅ GitHub installation successful!")
        
        # Step 4: Test imports from fresh installation
        print("\n🔍 Step 4: Testing imports after fresh GitHub install...")
        
        # Try importing directly
        try:
            from viincci_rag import ConfigManager, RAGSystem, UniversalResearchSpider
            print("✅ Successfully imported from viincci_rag:")
            print(f"   - ConfigManager: {ConfigManager.__module__}")
            print(f"   - RAGSystem: {RAGSystem.__module__}")
            print(f"   - UniversalResearchSpider: {UniversalResearchSpider.__module__}")
            
            # Check if they're viincci_rag or V4 classes
            if 'viincci_rag' in ConfigManager.__module__:
                print("\n✅ ALL CLASSES RESOLVE TO viincci_rag NAMESPACE (canonical path)")
            else:
                print("\n⚠️  Classes resolve to V4 namespace (fallback path)")
                
        except Exception as e:
            print(f"❌ Import test failed: {e}")
            
    else:
        print(f"❌ GitHub installation failed!")
        print(f"   Error: {result.stderr or result.stdout}")
        
except subprocess.TimeoutExpired:
    print("❌ Installation timed out (exceeded 120 seconds)")
except Exception as e:
    print(f"❌ Installation test error: {e}")
finally:
    # Cleanup temp directory
    if os.path.exists(test_env_dir):
        try:
            shutil.rmtree(test_env_dir)
            print(f"\n🧹 Cleaned up test directory")
        except:
            pass

print("\n" + "=" * 70)
print("INSTALLATION TEST COMPLETE - Ready for examples below")
print("=" * 70)

## 1. Setup: Imports and Mock SerpAPI

In [ ]:
# Main imports: Load core modules (viincci_rag primary, V4 fallback)
import os
import sys

print("\n" + "=" * 70)
print("LOADING CORE MODULES")
print("=" * 70)

def load_core_modules():
    """Try to import core modules from viincci_rag (primary), fall back to V4 (legacy).
    
    Returns: dict of {module_name: class_object} if successful, empty dict otherwise.
    """
    # Try new package layout first
    try:
        print("\n🔄 Attempting import from viincci_rag...")
        from viincci_rag import (
            ConfigManager,
            RAGSystem,
            UniversalResearchSpider,
            UniversalArticleGenerator
        )
        print('✅ Loaded core modules from viincci_rag (canonical import path)')
        print(f'   ConfigManager: {ConfigManager.__module__}')
        print(f'   RAGSystem: {RAGSystem.__module__}')
        print(f'   UniversalResearchSpider: {UniversalResearchSpider.__module__}')
        print(f'   UniversalArticleGenerator: {UniversalArticleGenerator.__module__}')
        return {
            'ConfigManager': ConfigManager,
            'UniversalResearchSpider': UniversalResearchSpider,
            'RAGSystem': RAGSystem,
            'UniversalArticleGenerator': UniversalArticleGenerator
        }
    except Exception as e_new:
        print(f'⚠️  Could not import from viincci_rag: {e_new}')
        
        # Try legacy package name used in older releases
        try:
            print("\n🔄 Attempting fallback import from V4...")
            from V4 import ConfigManager, UniversalResearchSpider, RAGSystem, UniversalArticleGenerator
            print('✅ Loaded core modules from V4 (legacy layout - backwards compatible)')
            print(f'   ConfigManager: {ConfigManager.__module__}')
            print(f'   RAGSystem: {RAGSystem.__module__}')
            print(f'   UniversalResearchSpider: {UniversalResearchSpider.__module__}')
            print(f'   UniversalArticleGenerator: {UniversalArticleGenerator.__module__}')
            return {
                'ConfigManager': ConfigManager,
                'UniversalResearchSpider': UniversalResearchSpider,
                'RAGSystem': RAGSystem,
                'UniversalArticleGenerator': UniversalArticleGenerator
            }
        except Exception as e_legacy:
            print(f'❌ Could not import from V4 either: {e_legacy}')
            print('⚠️  Continuing with safe mocks for demonstration.')
            return {}

viincci_modules = load_core_modules()

# Expose names into notebook globals if real modules were found
if viincci_modules:
    globals().update(viincci_modules)
    print("\n✅ All modules loaded and available for use below")
else:
    print("\n⚠️  Will use mock implementations for examples")

print("\n" + "=" * 70)

In [ ]:
# A tiny SerpAPI mock — returns a list of dicts with text and metadata
class MockSerpAPI:
    def __init__(self, key=None):
        self.key = key
    
    def search(self, query, num_results=3):
        """Return deterministic mock results based on the query string."""
        mock_results = []
        for i in range(num_results):
            mock_results.append({
                'text': f"Mock result {i+1} for '{query}'. This is a simulated search result.",
                'title': f"Mock Title {i+1} - {query.split()[0]}",
                'link': f"http://mock.example.com/{query.replace(' ', '-')}-{i+1}"
            })
        return mock_results

def get_serp_api_client():
    """Prefer real SerpAPI client if key and package available, else mock."""
    key = os.environ.get('SERP_API_KEY')
    if key:
        try:
            # Lazy import of serpapi client if available
            from serpapi import GoogleSearch
            print("✅ Using real SerpAPI client.")
            return GoogleSearch({'api_key': key})
        except ImportError:
            print("⚠️  SerpAPI client not installed, falling back to mock.")
            return MockSerpAPI(key)
        except Exception as e:
            print(f"⚠️  Error initializing SerpAPI client: {e}, falling back to mock.")
            return MockSerpAPI(key)
    else:
        print("ℹ️  SERP_API_KEY not found, using MockSerpAPI.")
        return MockSerpAPI()

serp_api = get_serp_api_client()

## Test Results Summary

**What was tested:**
1. ✅ Fresh GitHub installation: `pip install git+https://github.com/viincci/Viincci-RAG.git`
2. ✅ viincci_rag imports: `from viincci_rag import ConfigManager, RAGSystem, ...`
3. ✅ Namespace resolution: Verifies classes resolve to viincci_rag (not V4 fallback)

**Expected outcome:** All classes should resolve to `viincci_rag.core.*` namespace, indicating the canonical import path is working correctly.

**Backward compatibility:** If viincci_rag imports fail, the notebook automatically falls back to V4 imports (legacy support).

## 2. Helper Functions

In [ ]:
def normalize_search_results(results):
    """Convert search results into the shape expected by RAG systems."""
    normalized = []
    for r in results:
        normalized.append({
            'text': r.get('text', ''),
            'metadata': {
                'title': r.get('title'),
                'link': r.get('link'),
                **r.get('metadata', {})
            }
        })
    return normalized

# Example queries across different domains
queries = {
    'literature': 'Edgar Allan Poe poetry style themes Gothic',
    'botany': 'photosynthesis process details plant cells',
    'carpentry': 'wood joinery techniques types dovetail mortise tenon'
}

print(f"✅ Setup complete. Queries ready: {list(queries.keys())}")

## 3. Example: Mock Search Results

In [ ]:
# Run a mock search and display normalized results
query = queries['literature']
print(f"\n🔍 Searching: {query}")
results = serp_api.search(query, num_results=3)
normalized = normalize_search_results(results)

print(f"\n✅ Found {len(normalized)} results:")
for i, result in enumerate(normalized, 1):
    print(f"\n  [{i}] {result['metadata']['title']}")
    print(f"      Link: {result['metadata']['link']}")
    print(f"      Text: {result['text'][:100]}...")

## 4. Example: Using Real Modules (if available)

In [ ]:
# If real modules were imported successfully, you can use them here
if viincci_modules:
    print("✅ Real viincci_rag modules are available!")
    print(f"   Available: {list(viincci_modules.keys())}")
    # Example: 
    # config = ConfigManager()
    # spider = UniversalResearchSpider(config)
    # rag = RAGSystem(config)
    # article = UniversalArticleGenerator(config)
else:
    print("⚠️  Real modules not available. Using mock functions for demonstration.")

## 5. Additional Examples (Botany & Carpentry)

In [ ]:
# Example 2: Botany query
query = queries['botany']
print(f"\n🔍 Searching: {query}")
results = serp_api.search(query, num_results=2)
for i, result in enumerate(results, 1):
    print(f"\n  [{i}] {result['title']}")
    print(f"      {result['text'][:80]}...")

In [ ]:
# Example 3: Carpentry query
query = queries['carpentry']
print(f"\n🔍 Searching: {query}")
results = serp_api.search(query, num_results=2)
for i, result in enumerate(results, 1):
    print(f"\n  [{i}] {result['title']}")
    print(f"      {result['text'][:80]}...")

## Notes

- This notebook prioritizes `viincci_rag` imports but gracefully falls back to `V4` for backwards compatibility
- All searches use mock data by default (no API credits consumed)
- To use real SerpAPI, set the `SERP_API_KEY` environment variable and ensure `serpapi` is installed
- For real article generation, ensure your environment has the actual viincci_rag package installed

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/viincci/Viincci-RAG/blob/main/viincci_rag/examples/test_examples.ipynb)

# Viincci-RAG — Minimal Examples and Safe SerpAPI Mock

Run this notebook in **Google Colab** (link above) or locally with Jupyter.

**Features:**
- Backwards-compatible imports (viincci_rag → V4 fallback)
- SerpAPI mock to avoid using credits while testing
- Multiple domain examples (botany, literature, carpentry)
- Safe fallback generation when library unavailable

# Viincci-RAG — Minimal Examples and Safe SerpAPI Mock

This notebook provides a small, safe set of example cells to:

- Prefer imports from `viincci_rag` (new layout) and fall back to the legacy `V4` package when needed (backwards compatible).
- Demonstrate a SerpAPI mock so you can run examples without consuming real SerpAPI credits (useful for the free 250 credits/month constraint).
- Provide tiny generation examples across several domains using either the real library (if installed) or a harmless fallback that stitches mock research results together.

Notes: Run cells sequentially. If you have the real library installed and a valid SerpAPI key configured, the notebook will attempt to use it; otherwise it uses safe mocks.

In [ ]:
# Backwards-compatible imports: prefer viincci_rag, fallback to V4 (legacy)
import importlib

,
def try_import():
    globals_local = {}
    # Try new package layout first
    try:
        # attempt to import the new package layout (viincci_rag)
        from viincci_rag.core.config import ConfigManager
        from viincci_rag.core.spider import UniversalResearchSpider
        from viincci_rag.core.rag_system import RAGSystem
        from viincci_rag.core.article_generator import UniversalArticleGenerator
        print('Imported core modules from viincci_rag (new layout)')
        return {
            'ConfigManager': ConfigManager,
            'UniversalResearchSpider': UniversalResearchSpider,
            'RAGSystem': RAGSystem,
            'UniversalArticleGenerator': UniversalArticleGenerator
        }
    except Exception as e_new:
        # Try legacy package name used in older releases
        try:
            from V4 import ConfigManager, UniversalResearchSpider, RAGSystem, UniversalArticleGenerator
            print('Imported core modules from V4 (legacy layout)')
            return {
                'ConfigManager': ConfigManager,
                'UniversalResearchSpider': UniversalResearchSpider,
                'RAGSystem': RAGSystem,
                'UniversalArticleGenerator': UniversalArticleGenerator
            }
        except Exception as e_legacy:
            print('Could not import core modules from viincci_rag or V4. Continuing with safe mocks.')
            return {}
viincci_modules = try_import()
# Expose names into notebook globals if real modules were found
if viincci_modules:
    globals().update(viincci_modules)
: 
,
: {
: 

: [
,
,

: 
,
: {
: 

: [
,
# A tiny SerpAPI mock — returns a list of 
 dicts with text and metadata
class MockSerpAPI:
    def __init__(self, key=None):
        self.key = key
    def search(self, query, num_results=3):
        # Deterministic mock results based on the query string
        return [
    # Prefer a real SerpAPI client if the key is present and the package is installed
    key = os.environ.get('SERP_API_KEY')
    if key:
        try:
            # lazy import of serpapi client if available in the environment
            from serpapi import GoogleSearch
            return GoogleSearch({'api_key': key})
        except Exception:
,
    else:
,

In [ ]:
# Small helper to convert search results into the shape expected by RAG systems in this repo
def normalize_search_results(results):
    normalized = []
    for r in results:
        normalized.append({'text': r.get('text', ''), 'metadata': {'title': r.get('title'), **r.get('metadata', {})}})
,
# Example: run safe searches and produce short generated outputs using either the real generators or a fallback
queries = {
    'literature': 'Edgar Allan Poe poetry style themes Gothic',